In [16]:
from dataloader.FireSpreadDataset import FireSpreadDataset

dataset_with_doy = FireSpreadDataset(
    data_dir="/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/Data/WIldfireSpreadTS_HDF5",
    included_fire_years=[2019],
    n_leading_observations=5,
    crop_side_length=128,
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=(2018, 2020),
    return_doy=True  # ✅ get day-of-year
)


In [17]:
from dataloader.FireSpreadDataset import FireSpreadDataset

dataset_without_doy = FireSpreadDataset(
    data_dir="/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/Data/WIldfireSpreadTS_HDF5",
    included_fire_years=[2019],
    n_leading_observations=5,
    crop_side_length=128,
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=(2018, 2020),
    return_doy=False # ✅ get day-of-year
)


In [18]:
# Dataset with DOY
x1, y1, doys1 = dataset_with_doy[0]
print("Sample with DOY:", doys1)

# Dataset without DOY just gives x,y
x2, y2 = dataset_without_doy[0]
print("Shape without DOY:", x2.shape)


Sample with DOY: tensor([66., 67., 68., 69., 70.])
Shape without DOY: torch.Size([5, 40, 128, 128])


In [15]:
from models.SMPTempModel import SMPTempModel

model_with_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x1.shape[1], 
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",  # ✅ required!
    use_doy=True
)


model_no_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x1.shape[1],
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",  # ✅
    use_doy=False
)


# (B, T, C, H, W) inputs
x1_batched = x1.unsqueeze(0)
doys1_batched = doys1.unsqueeze(0)

# Run through both models
out_with_doy = model_with_doy(x1_batched, doys1_batched)
out_no_doy = model_no_doy(x1_batched, doys1_batched)

diff = torch.mean(torch.abs(out_with_doy - out_no_doy)).item()
print(f"🚀 Mean absolute diff between DOY vs no DOY: {diff:.6f}")


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/sameeps/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 11.1MB/s]


Loaded resnet18 with imagenet weights + LTAE
Loaded resnet18 with imagenet weights + LTAE
🚀 Mean absolute diff between DOY vs no DOY: 1.067734


In [30]:
import torch
from dataloader.FireSpreadDataset import FireSpreadDataset
from models.SMPTempModel import SMPTempModel

# ✅ Load your dataset
dataset_with_doy = FireSpreadDataset(
    data_dir="/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/Data/WIldfireSpreadTS_HDF5",
    included_fire_years=[2021],
    n_leading_observations=5,
    crop_side_length=128,
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=[2018, 2020],
    return_doy=True
)

# ✅ Grab one sample
x1, y1, doys1 = dataset_with_doy[0]
print(f"✅ Loaded sample shapes:\n"
      f"    x:    {x1.shape}  # (T, C, H, W)\n"
      f"    y:    {y1.shape}  # (H, W)\n"
      f"    doys: {doys1.shape}  # (T,) \n "
      f" doys values: {doys1}"
      )


# ✅ Add batch dimension
x1_batched = x1.unsqueeze(0)       # (1, T, C, H, W)
doys1_batched = doys1.unsqueeze(0) # (1, T)

# ✅ Make sure these are float tensors
x1_batched = x1_batched.float()
doys1_batched = doys1_batched.float()

print(f"\n🚀 Ready to pass through model:\n"
      f"    x1_batched:    {x1_batched.shape}\n"
      f"    doys1_batched: {doys1_batched.shape}")

# ✅ Instantiate the model
model_with_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x1.shape[1],  # C
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",
    use_doy=True
)

model_no_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x1.shape[1],  # C
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",
    use_doy=False
)

# ✅ Forward pass with DOY
print("\n>>> Running forward with DOY...")
out_with_doy = model_with_doy.forward(x1_batched, doys1_batched)
print(f"✅ Output shape (with DOY): {out_with_doy.shape}")

# ✅ Forward pass without DOY
print("\n>>> Running forward without DOY...")
out_no_doy = model_no_doy.forward(x1_batched, doys1_batched)
print(f"✅ Output shape (without DOY): {out_no_doy.shape}")


✅ Loaded sample shapes:
    x:    torch.Size([5, 40, 128, 128])  # (T, C, H, W)
    y:    torch.Size([128, 128])  # (H, W)
    doys: torch.Size([5])  # (T,) 
  doys values: tensor([33., 34., 35., 36., 37.])

🚀 Ready to pass through model:
    x1_batched:    torch.Size([1, 5, 40, 128, 128])
    doys1_batched: torch.Size([1, 5])
Loaded resnet18 with imagenet weights + LTAE
Loaded resnet18 with imagenet weights + LTAE

>>> Running forward with DOY...
✅ Output shape (with DOY): torch.Size([1, 1, 128, 128])

>>> Running forward without DOY...
✅ Output shape (without DOY): torch.Size([1, 1, 128, 128])


In [32]:
import torch
from dataloader.FireSpreadDataset import FireSpreadDataset
from models.SMPTempModel import SMPTempModel

# ✅ Load dataset that does NOT return DOY
dataset_no_doy = FireSpreadDataset(
    data_dir="/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/Data/WIldfireSpreadTS_HDF5",
    included_fire_years=[2019],
    n_leading_observations=5,
    crop_side_length=128,
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=[2018, 2020],
    return_doy=False  # <---- no doy returned
)

# ✅ Grab a single sample
x2, y2 = dataset_no_doy[0]
print(f"\n✅ Loaded sample WITHOUT DOY:\n"
      f"    x shape: {x2.shape}  # (T, C, H, W)\n"
      f"    y shape: {y2.shape}  # (H, W)")

# ✅ Add batch dimension
x2_batched = x2.unsqueeze(0).float()  # (1, T, C, H, W)
print(f"\n🚀 x2_batched shape (ready for model): {x2_batched.shape}")

# ✅ Instantiate your model
model_with_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x2.shape[1],  # C
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",
    use_doy=True  # IMPORTANT: model expects to use DOY, will auto-create
)

# ✅ Forward pass without giving DOY (will create dummy DOY internally)
print("\n>>> Running forward on model with use_doy=True but doy=None:")
out_auto_doy = model_with_doy.forward(x2_batched, None)
print(f"✅ Output shape (auto dummy DOY created inside forward): {out_auto_doy.shape}")

# ✅ Extra: you can also force `use_doy=False` to see different behavior
model_no_doy = SMPTempModel(
    encoder_name="resnet18",
    n_channels=x2.shape[1],
    flatten_temporal_dimension=False,
    pos_class_weight=1.0,
    encoder_weights="imagenet",
    loss_function="Dice",
    use_doy=False  # model does NOT use DOY at all
)

print("\n>>> Running forward on model with use_doy=False (also creates dummy internally):")
out_forced_no_doy = model_no_doy.forward(x2_batched, None)
print(f"✅ Output shape (use_doy=False so dummy DOY used anyway): {out_forced_no_doy.shape}")



✅ Loaded sample WITHOUT DOY:
    x shape: torch.Size([5, 40, 128, 128])  # (T, C, H, W)
    y shape: torch.Size([128, 128])  # (H, W)

🚀 x2_batched shape (ready for model): torch.Size([1, 5, 40, 128, 128])
Loaded resnet18 with imagenet weights + LTAE

>>> Running forward on model with use_doy=True but doy=None:
✅ Output shape (auto dummy DOY created inside forward): torch.Size([1, 1, 128, 128])
Loaded resnet18 with imagenet weights + LTAE

>>> Running forward on model with use_doy=False (also creates dummy internally):
✅ Output shape (use_doy=False so dummy DOY used anyway): torch.Size([1, 1, 128, 128])


In [13]:
import torch

# === CONFIGURATION ===
ckpt_path = "/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/src/models/utae_paps_models/Gallelio-weights/seco_resnet18_1m.ckpt"               # input .ckpt file path
# output_full_model_path = "seco_resnet18_full.pth" # will contain full state_dict
output_encoder_path = "/Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/src/models/utae_paps_models/Gallelio-weights/seco_resnet18_1m.pth" # will contain encoder-only weights

# === LOAD CHECKPOINT ===
checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)

# Handle both old and raw state_dict formats
if "state_dict" in checkpoint:
    state_dict = checkpoint["state_dict"]
else:
    state_dict = checkpoint  # raw dict

# === SAVE FULL MODEL IN COMPATIBLE FORMAT ===
# torch.save({"state_dict": state_dict}, output_full_model_path)
# print(f"✅ Full model state_dict saved to {output_full_model_path}")

# === OPTIONALLY SAVE ENCODER-ONLY VERSION ===
encoder_prefix = "encoder."
encoder_state_dict = {
    k[len(encoder_prefix):]: v
    for k, v in state_dict.items()
    if k.startswith(encoder_prefix)
}

torch.save({"state_dict": encoder_state_dict}, output_encoder_path)
print(f"✅ Encoder-only state_dict saved to {output_encoder_path}")


✅ Encoder-only state_dict saved to /Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/src/models/utae_paps_models/Gallelio-weights/seco_resnet18_1m.pth


In [15]:
# Load the checkpoint
checkpoint = torch.load(ckpt_path, map_location='cpu',weights_only=False)

# Get actual state_dict
state_dict = checkpoint["state_dict"] if "state_dict" in checkpoint else checkpoint

# Check all keys (print first 20)
print("First 20 keys in state_dict:")
for i, key in enumerate(state_dict):
    print(f"{i+1:2d}: {key}")
    if i == 19:
        break


First 20 keys in state_dict:
 1: queue
 2: queue_ptr
 3: encoder_q.0.weight
 4: encoder_q.1.weight
 5: encoder_q.1.bias
 6: encoder_q.1.running_mean
 7: encoder_q.1.running_var
 8: encoder_q.1.num_batches_tracked
 9: encoder_q.4.0.conv1.weight
10: encoder_q.4.0.bn1.weight
11: encoder_q.4.0.bn1.bias
12: encoder_q.4.0.bn1.running_mean
13: encoder_q.4.0.bn1.running_var
14: encoder_q.4.0.bn1.num_batches_tracked
15: encoder_q.4.0.conv2.weight
16: encoder_q.4.0.bn2.weight
17: encoder_q.4.0.bn2.bias
18: encoder_q.4.0.bn2.running_mean
19: encoder_q.4.0.bn2.running_var
20: encoder_q.4.0.bn2.num_batches_tracked


In [16]:
import torch

# ckpt_path = "seco_resnet18_1m.ckpt"
# output_encoder_path = "seco_resnet18_encoder.pth"

checkpoint = torch.load(ckpt_path, map_location='cpu', weights_only=False)
state_dict = checkpoint["state_dict"] if "state_dict" in checkpoint else checkpoint

# Updated prefix
prefix = "encoder_q."
encoder_state_dict = {
    k[len(prefix):]: v
    for k, v in state_dict.items()
    if k.startswith(prefix)
}

print(f"✅ Found {len(encoder_state_dict)} encoder_q params")

# Save wrapped in {"state_dict": ...} to work with your load_checkpoint()
torch.save({"state_dict": encoder_state_dict}, output_encoder_path)
print(f"✅ Encoder weights saved to {output_encoder_path}")


✅ Found 120 encoder_q params
✅ Encoder weights saved to /Users/sameeps/Documents/Dr-Malof/WIldfire/WildfireSpreadTS/src/models/utae_paps_models/Gallelio-weights/seco_resnet18_1m.pth
